In [70]:
import carla
import numpy as np
client = carla.Client('localhost', 2000)
client.set_timeout(2.0)

In [71]:
client.set_timeout(30)

In [72]:
client.load_world("Town01")

In [73]:
import random, re
world_map = client.get_world().get_map()
client.get_world().set_weather(find_weather_presets()[2][0])
blueprint_library = client.get_world().get_blueprint_library()
spawn_pts = world_map.get_spawn_points()
spawn_point = random.choice(spawn_pts)

In [74]:
vehicle_blueprint = blueprint_library.filter("model3")[0]

In [75]:
vehicle = client.get_world().spawn_actor(vehicle_blueprint, spawn_point)

In [112]:
transform = vehicle.get_transform()

In [113]:
get_matrix(transform)

matrix([[-5.32630710e-07,  1.00000000e+00, -1.75757499e-05,
          9.27982788e+01],
        [-9.99997168e-01, -4.90798723e-07,  2.38001099e-03,
          1.16544640e+02],
        [ 2.38001098e-03,  1.75769678e-05,  9.99997168e-01,
         -5.28194429e-03],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+00]])

In [114]:
create_bb_points(vehicle)

array([[ 2.40441084,  1.08475292, -0.76159108,  1.        ],
       [-2.40441084,  1.08475292, -0.76159108,  1.        ],
       [-2.40441084, -1.08475292, -0.76159108,  1.        ],
       [ 2.40441084, -1.08475292, -0.76159108,  1.        ],
       [ 2.40441084,  1.08475292,  0.76159108,  1.        ],
       [-2.40441084,  1.08475292,  0.76159108,  1.        ],
       [-2.40441084, -1.08475292,  0.76159108,  1.        ],
       [ 2.40441084, -1.08475292,  0.76159108,  1.        ]])

In [115]:
coords = np.matmul(get_matrix(transform), create_bb_points(vehicle).T).T
coords

matrix([[ 93.88304383, 114.13842243,  -0.76112927,   1.        ],
        [ 93.88304639, 118.94723049,  -0.77257432,   1.        ],
        [ 91.71354056, 118.94723155,  -0.77261245,   1.        ],
        [ 91.713538  , 114.1384235 ,  -0.76116741,   1.        ],
        [ 93.88301706, 114.14204762,   0.76204857,   1.        ],
        [ 93.88301962, 118.95085568,   0.75060352,   1.        ],
        [ 91.71351379, 118.95085674,   0.75056538,   1.        ],
        [ 91.71351123, 114.14204869,   0.76201043,   1.        ]])

In [116]:
tmp_location = coords[0][:3]
tmp_location[0,1]

114.13842243063641

In [117]:
a = carla.Location()
a.x = tmp_location[0,0]
a.y = tmp_location[0,1]
a.z = tmp_location[0,2]


tmp_location = coords[1][:3]

b = carla.Location()
b.x = tmp_location[0,0]
b.y = tmp_location[0,1]
b.z = tmp_location[0,2]

tmp_location = coords[2][:3]

c = carla.Location()
c.x = tmp_location[0,0]
c.y = tmp_location[0,1]
c.z = tmp_location[0,2]

tmp_location = coords[3][:3]

d = carla.Location()
d.x = tmp_location[0,0]
d.y = tmp_location[0,1]
d.z = tmp_location[0,2]

In [123]:
# client.get_world().debug.draw_string(a, '0', draw_shadow=False,
#                             color=carla.Color(r=255, g=0, b=0), life_time=5,
#                             persistent_lines=False)
# client.get_world().debug.draw_string(b, '0', draw_shadow=False,
#                             color=carla.Color(r=255, g=0, b=0), life_time=5,
#                             persistent_lines=False)
# client.get_world().debug.draw_string(c, '0', draw_shadow=False,
#                             color=carla.Color(r=255, g=0, b=0), life_time=5,
#                             persistent_lines=False)
client.get_world().debug.draw_string(d, '0', draw_shadow=False,
                            color=carla.Color(r=255, g=0, b=0), life_time=5,
                            persistent_lines=False)

In [87]:
vehicle.get_location().x, vehicle.get_location().y

(92.79827880859375, 116.54463958740234)

In [32]:
def get_matrix(transform):
    """
    Creates matrix from carla transform.
    """

    rotation = transform.rotation
    location = transform.location
    c_y = np.cos(np.radians(rotation.yaw))
    s_y = np.sin(np.radians(rotation.yaw))
    c_r = np.cos(np.radians(rotation.roll))
    s_r = np.sin(np.radians(rotation.roll))
    c_p = np.cos(np.radians(rotation.pitch))
    s_p = np.sin(np.radians(rotation.pitch))
    matrix = np.matrix(np.identity(4))
    matrix[0, 3] = location.x
    matrix[1, 3] = location.y
    matrix[2, 3] = location.z
    matrix[0, 0] = c_p * c_y
    matrix[0, 1] = c_y * s_p * s_r - s_y * c_r
    matrix[0, 2] = -c_y * s_p * c_r - s_y * s_r
    matrix[1, 0] = s_y * c_p
    matrix[1, 1] = s_y * s_p * s_r + c_y * c_r
    matrix[1, 2] = -s_y * s_p * c_r + c_y * s_r
    matrix[2, 0] = s_p
    matrix[2, 1] = -c_p * s_r
    matrix[2, 2] = c_p * c_r
    return matrix



def create_bb_points(vehicle):
    """
    Returns 3D bounding box for a vehicle.
    """

    cords = np.zeros((8, 4))
    extent = vehicle.bounding_box.extent
    cords[0, :] = np.array([extent.x, extent.y, -extent.z, 1])
    cords[1, :] = np.array([-extent.x, extent.y, -extent.z, 1])
    cords[2, :] = np.array([-extent.x, -extent.y, -extent.z, 1])
    cords[3, :] = np.array([extent.x, -extent.y, -extent.z, 1])
    cords[4, :] = np.array([extent.x, extent.y, extent.z, 1])
    cords[5, :] = np.array([-extent.x, extent.y, extent.z, 1])
    cords[6, :] = np.array([-extent.x, -extent.y, extent.z, 1])
    cords[7, :] = np.array([extent.x, -extent.y, extent.z, 1])
    return cords

def find_weather_presets():
    rgx = re.compile('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)')
    name = lambda x: ' '.join(m.group(0) for m in rgx.finditer(x))
    presets = [x for x in dir(carla.WeatherParameters) if re.match('[A-Z].+', x)]
    return [(getattr(carla.WeatherParameters, x), name(x)) for x in presets]
